# Prepatation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from utils import ModelAndTokenizer, plot_scores, find_token_range
import matplotlib.pyplot as plt
import numpy as np
from causal_trace import plot_all_flow
import os, re, json
torch.set_grad_enabled(False)

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"  
device = "cuda:7"
mt = ModelAndTokenizer(model_name, device)

# Case1 (Country) (Capital,Currency,Area,Population,Religion,Language)

## 1.Location by CT

In [ ]:
id = 8
data_path = "/home/zijianwang/ACL2024/country_capital_city.json"
with open(data_path, 'r') as file:
    data = json.load(file)
Template = data["Prompt"]
relation = data["Relation"]
subject = data["samples"][id]["subject"]
object = data["samples"][id]["object"]
print(f"Subject: {subject}, Relation: {relation}, Object: {object}")
print(Template)

In [ ]:
Recall_prompt = Template.format(subject)
token_range_sub = find_token_range(mt.tokenizer, Recall_prompt, subject)
token_range_rel = find_token_range(mt.tokenizer, Recall_prompt, relation)
# token_range_rel[1] = token_range_rel[1] + 3

res_dict_no_rel = plot_all_flow(mt, Recall_prompt, subject = None, noise = 1, type = None, noise_range = token_range_rel)
print("--------------------------------------------------")
res_dict_no_sub  = plot_all_flow(mt, Recall_prompt, subject = None, noise = 1, type = None, noise_range = token_range_sub)


In [ ]:
score_no_rel = res_dict_no_rel["hs"]['scores'][-1]
score_no_sub = res_dict_no_sub["hs"]['scores'][-1]
# score_no_instructs.append(score_no_instruct)
# score_no_inputs.append(score_no_input)
title = f"Mediating effect of subject and relation at last token, {Recall_prompt}"
plot_scores(score_no_rel, score_no_sub, title)

In [ ]:
diff = torch.abs(score_no_rel - score_no_sub)
threshold = 0.4
big_diff_mask = diff > threshold
start_idx = torch.where(big_diff_mask)[0][0] if torch.any(big_diff_mask) else None
end_idx = torch.where(big_diff_mask)[0][-1]  if torch.any(big_diff_mask) else None
Range = (int(start_idx), int(end_idx))
if start_idx is None or end_idx is None:
    print("No Relation Emergence Stage Found (diff > threshold)!")
else:
    print(f"Relation Emergence Stage: {Range}")

## Zero shot

In [ ]:
from util.locating_extracting import zero_shot, edit

In [ ]:
acc_zs, acc_full, good_cases = zero_shot(mt, data_path, Recall_prompt, layer_range = Range)

In [ ]:
import pickle
goog_cases_path = "good_cases_{}.pkl".format(data["name"])
with open(goog_cases_path , 'wb') as f:
    pickle.dump(good_cases, f)


In [ ]:
good_data = pickle.load(open(goog_cases_path, "rb"))
print(good_data)
print(len(good_data))

In [ ]:
acc = edit(mt, good_data, Recall_prompt, relation, layer_range = Range)

In [ ]:
import numpy as np
import scipy.stats as st

# generate observed data
X = st.norm(loc=3, scale=1).rvs(size=1000)
print(X)

In [ ]:
import numpy as np
import scipy.stats as st

# generate observed data
X = st.norm(loc=3, scale=1).rvs(size=1000)
print(X)

def guassian_posterior(X, theta):
    # returns the unnormalized log posterior
    loglik = np.sum(np.log(st.norm(loc=theta, scale=1).pdf(X)))
    logprior = np.log(st.norm(loc=0, scale=1).pdf(theta))
    
    return loglik + logprior
    
def guassian_proposal(theta_curr):
    # proposal based on Gaussian
    theta_new = st.norm(loc=theta_curr, scale=0.2).rvs()
    return theta_new

def guassian_proposal_prob(x1, x2):
    # calculate proposal probability q(x2|x1), based on Gaussian
    q = st.norm(loc=x1, scale=1).pdf(x2)
    return q

def mcmc_mh_posterior(X, theta_init, func, proposal_func, proposal_func_prob, n_iter=1000):
    # Metropolis-Hastings to estimate posterior
    thetas = []
    theta_curr = theta_init
    accept_rates = []
    accept_cum = 0
    
    for i in range(1, n_iter+1):
        theta_new = proposal_func(theta_curr)
        
        prob_curr = func(X, theta_curr)
        prob_new = func(X, theta_new)
        
        # we calculate the prob=exp(x) only when prob<1 so the exp(x) will not overflow for large x
        if prob_new > prob_curr:
            acceptance_ratio = 1
        else:
            qr = proposal_func_prob(theta_curr, theta_new)/proposal_func_prob(theta_new, theta_curr)
            acceptance_ratio = np.exp(prob_new - prob_curr) * qr
        acceptance_prob = min(1, acceptance_ratio)
        
        if acceptance_prob > st.uniform(0,1).rvs():
            print("Accept")
            theta_curr = theta_new
            accept_cum = accept_cum+1
            thetas.append(theta_new)
        else:
            print("Reject")
            thetas.append(theta_curr)
            
        accept_rates.append(accept_cum/i)
        
    return thetas, accept_rates

# run MCMC
thetas, accept_rates = mcmc_mh_posterior(X, 1, 
                                         guassian_posterior, guassian_proposal, guassian_proposal_prob, 
                                         n_iter=8000)

In [ ]:
print(thetas)

In [ ]:
import numpy as np
import scipy.stats as st

# generate observed data from a mixture of two Gaussians
X = np.concatenate([
    st.norm(loc=10, scale=0.6).rvs(size=500),
    st.norm(loc=1, scale=0.7).rvs(size=500)
])

print(X)

def mixture_posterior(X, theta):
    # returns the unnormalized log posterior
    loglik1 = np.sum(np.log(st.norm(loc=theta[0], scale=1).pdf(X)))
    loglik2 = np.sum(np.log(st.norm(loc=theta[1], scale=1).pdf(X)))
    loglik = np.logaddexp(loglik1, loglik2) - np.log(2)
    logprior = np.sum(np.log(st.norm(loc=0, scale=3).pdf(theta)))
    return loglik + logprior

def mixture_proposal(theta_curr):
    # proposal based on Gaussian
    theta_new = st.multivariate_normal(mean=theta_curr, cov=0.5*np.eye(2)).rvs()
    return theta_new

def mixture_proposal_prob(x1, x2):
    # calculate proposal probability q(x2|x1), based on Gaussian
    q = st.multivariate_normal(mean=x1, cov=0.5*np.eye(2)).pdf(x2)
    return q

def mcmc_mh_posterior(X, theta_init, func, proposal_func, proposal_func_prob, n_iter=1000):
    # Metropolis-Hastings to estimate posterior
    thetas = []
    theta_curr = theta_init
    accept_rates = []
    accept_cum = 0

    for i in range(1, n_iter+1):
        theta_new = proposal_func(theta_curr)
        prob_curr = func(X, theta_curr)
        prob_new = func(X, theta_new)

        # we calculate the prob=exp(x) only when prob<1 so the exp(x) will not overflow for large x
        if prob_new > prob_curr:
            acceptance_ratio = 1
        else:
            qr = proposal_func_prob(theta_curr, theta_new)/proposal_func_prob(theta_new, theta_curr)
            acceptance_ratio = np.exp(prob_new - prob_curr) * qr

        acceptance_prob = min(1, acceptance_ratio)

        if acceptance_prob > st.uniform(0,1).rvs():
            print("Accept")
            theta_curr = theta_new
            accept_cum = accept_cum+1
        else:
            print("Reject")

        thetas.append(theta_curr)
        accept_rates.append(accept_cum/i)

    return thetas, accept_rates

# run MCMC
thetas, accept_rates = mcmc_mh_posterior(X, [0, 0],
                                         mixture_posterior, mixture_proposal, mixture_proposal_prob,
                                         n_iter=9000)

In [ ]:
print(thetas)